#### <center>Intermediate Python and Software Enginnering</center>


## <center>Section 08 - Part 01 - Concurrency and Thread</center>


### <center>Innovation Scholars Programme</center>
### <center>King's College London, Medical Research Council and UKRI <center>

## What we'll cover
* Native libraries
* concurrency
* threads, processes
* deadlock, locks, semaphores
* shared memory
* GIL
* GPU computation

# Concurrency

* Running multiple computational streams at the same time
* Seen when running multiple programs at same time on a device
* Each running program exists in a process, these can be executed in parallel
* Within each process can be smaller lightweight streams of control flow called threads
* GPUs use massive parallelism to render fast

* Simple thread example in Python:

In [1]:
%load_ext nb_black
import threading, time


def print_msg(msg):
    for i in range(5):
        print(time.time(), msg)
        time.sleep(0.1)


t = threading.Thread(target=print_msg, args=("Hello, thread!",))
t.start()
print(time.time(), "Done")

1584566359.368052 Hello, thread!
1584566359.3685863 Done


<IPython.core.display.Javascript object>

1584566359.4684453 Hello, thread!
1584566359.5687037 Hello, thread!
1584566359.6689537 Hello, thread!
1584566359.7692175 Hello, thread!


* In Python, a thread object is created which represents a separate flow of execution
* Target callable (or `Thread.run()` if this is `None`) is executed concurrently
* Thread calling `start()` can continue on its own after that
* Allows multiple computations to be done at once, using more CPU resources if multiple cores are present

* Python programs are often composed of multiple threads
* When running on command line or in Jupyter you interact with one directly, others in background could be doing other tasks
* Eg. in many GUI frameworks, one "message pump" thread handles events coming from user input and triggers code elsewhere to do some task
* If these task routines do not spawn new threads they will lock the windowing environment until they complete

* Thread objects have methods for control:
* `start` starts the thread, only call once
* `join` waits for the thread to finish before allowing the calling thread to continue (blocking)
* `is_alive` checks if the thread is still running

In [2]:
t = threading.Thread(target=print_msg, args=("Hello, thread!",))
t.start()
print(t.is_alive())  # is the thread running?
t.join()  # wait for thread to finish
print(time.time(), "Done", t.is_alive())

1584566365.8451903 Hello, thread!
True
1584566365.945567 Hello, thread!
1584566366.0458167 Hello, thread!
1584566366.1460655 Hello, thread!
1584566366.2463303 Hello, thread!
1584566366.3473692 Done False


<IPython.core.display.Javascript object>

* Threads need to co-operate to implement concurrent behaviour, special mechanisms must be used to do this and prevent semantic issues
* Eg. if a thread is stopped waiting for an event that never occurs, it is deadlocked
* Happens if two threads rely on the other to respond, can't do that while waiting
* Eg. if a data structure is manipulated by two threads at once these can overwrite one anothers changes and put the structure in an inconsistent state

* Simplest mechanism is a lock which only one thread can own at a time, other threads requesting the lock must wait:

In [3]:
lock = threading.Lock()


def do_critical():
    print(time.time(), "I want to do something!")
    with lock:  # critical section is execution area where lock is held
        print(time.time(), "Doing the thing!")


t = threading.Thread(target=do_critical)
with lock:
    t.start()
    print(time.time(), "No, you have to wait...")
    time.sleep(5)

1584566369.747519 I want to do something!
1584566369.7479808 No, you have to wait...
1584566374.7548938 Doing the thing!


<IPython.core.display.Javascript object>

* Ensures that only one thread can enter critical section at a time
* Used to co-ordinate access to a data structure, only code in a critical section should modify something shared
* Deadlock can still occur if one thread holds a lock another is waiting on and won't release until that one responds somehow; this can't happen so both are stuck
* Any behaviour dependent on timing is a race condition and can cause unwanted behaviour

* Famous thought experiment is Dining Philosophers <img style="float:right;margin:2cm" width="350" src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/7b/An_illustration_of_the_dining_philosophers_problem.png/463px-An_illustration_of_the_dining_philosophers_problem.png">
* 5 people at table with 5 forks, need 2 forks to eat (ie. acquire fork locks)
* If all 5 pick up a fork at once, deadlock
* Each diner must try to pick up one fork then another, if this fails they have to let go of a fork to let others continue